In [10]:
### IMPORTS ###
from mmap import mmap, ACCESS_READ
from struct import unpack
from math import sqrt
from glob import glob 
from os import chdir, listdir, makedirs
from os.path import splitext, isdir, exists
import numpy as np
import time
import pandas as pd

In [13]:
# b'\x00\x01\xc4\xb9\xff\xcb\xfb\xe4\x00\x83\x0b'
def unpacking_v2_format_hig(row):
 
    list_values = []
    #index = unpack('>I', bytes(row[0:4]))[0]
    acc_x = (unpack('>h', bytes(row[4:6]))[0])  / 1024
    acc_y = (unpack('>h', bytes(row[6:8]))[0])  / 1024
    acc_z = (unpack('>h', bytes(row[8:10]))[0]) / 1024

    #index = (index / 6000)
    #list_values.append(index)
    acc_x = float(acc_x)
    list_values.append(acc_x) 
    acc_y = float(acc_y) 
    list_values.append(acc_y)
    acc_z = float(acc_z)
    list_values.append(acc_z)
    amag = round(sqrt(pow(acc_x, 2) + pow(acc_y, 2) + pow(acc_z, 2)),2)
    list_values.append(amag)
    return list_values

def read_row(mm,hig=1):
    count = 0
    while True:
        count += 1
        if(hig==1):
            row = mm.read(11) # packet length
            if not len(row) == 11: 
                break 
        yield row 

def get_results_v2_format(hig_filename):
    complete_results = []    
    if hig_filename:
        print("getting results for", hig_filename)
        with open(hig_filename,'rb') as file:
            mm = mmap(file.fileno(), 0, access=ACCESS_READ)
            results = [unpacking_v2_format_hig(row) for row in read_row(mm)]
            complete_results.append(results)
            mm.close()
            file.close() 
        
    print("completed conversion",flush=True)
    return complete_results

In [14]:
#hig_filename = "data/mc_5_reps_test_1.HIG"
hig_filename = "data/mc_test_2_60bpm.HIG"
#hig_filename = "data/mc_test_3_ml_60bpm.HIG"
#hig_filename = "data/lc_vag_test_1_60bpm.HIG"

converted_data = get_results_v2_format(hig_filename)
print(converted_data[0][0:10])

getting results for data/mc_test_2_60bpm.HIG
completed conversion
[[-0.087890625, -0.603515625, 0.2265625, 0.65], [-0.087890625, -0.6064453125, 0.2275390625, 0.65], [-0.0888671875, -0.611328125, 0.228515625, 0.66], [-0.0888671875, -0.6142578125, 0.2294921875, 0.66], [-0.0888671875, -0.6171875, 0.23046875, 0.66], [-0.08984375, -0.619140625, 0.2314453125, 0.67], [-0.08984375, -0.6220703125, 0.2333984375, 0.67], [-0.08984375, -0.6240234375, 0.234375, 0.67], [-0.08984375, -0.626953125, 0.234375, 0.68], [-0.0908203125, -0.6298828125, 0.2353515625, 0.68]]


In [15]:
# write to csv
columns = ['x', 'y', 'z', 'mag']

# Create the DataFrame
df = pd.DataFrame(converted_data[0], columns=columns)
print(df.head(5))

df.to_csv("mc_test_2_60bpm.csv", index=False)

          x         y         z   mag
0 -0.087891 -0.603516  0.226562  0.65
1 -0.087891 -0.606445  0.227539  0.65
2 -0.088867 -0.611328  0.228516  0.66
3 -0.088867 -0.614258  0.229492  0.66
4 -0.088867 -0.617188  0.230469  0.66
